In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from datetime import datetime
import warnings
import win32com.client as win32
warnings.filterwarnings('ignore')
import pymssql
import psycopg2
from sqlalchemy import create_engine
pd.set_option('display.max_column', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
query = """SELECT 
    dbo.SRA010.RA_FILIAL AS Filial, 
    dbo.SRA010.RA_MAT AS Matricula, 
    dbo.SRA010.RA_NOME AS Funcionário, 
    FORMAT(CONVERT(DATE, dbo.SRA010.RA_NASC), 'dd/MM/yyyy') AS Nascimento, 
    FORMAT(CONVERT(DATE, dbo.SRA010.RA_ADMISSA), 'dd/MM/yyyy') AS Admissão, 
    FORMAT(CONVERT(DATE, dbo.SRA010.RA_DEMISSA), 'dd/MM/yyyy') AS Desligamento, 
    dbo.SRA010.RA_SALARIO AS Salário, 
    CASE dbo.SRA010.RA_ADCINS 
        WHEN '1' THEN 'Não' 
        WHEN '2' THEN 'Mínima' 
        WHEN '3' THEN 'Média' 
        WHEN '4' THEN 'Máxima' 
        ELSE 'Não' 
    END AS Insalubridade, 
    CASE dbo.SRA010.RA_ADCPERI 
        WHEN '1' THEN 'Não' 
        ELSE 'Sim' 
    END AS Periculosidade, 
    dbo.SRA010.RA_RG AS RG, 
    FORMAT(CONVERT(DATE, dbo.SRA010.RA_DTRGEXP), 'dd/MM/yyyy') AS [Exp RG], 
    dbo.SRA010.RA_ORGEMRG AS [Org Em RG], 
    dbo.SRA010.RA_RGUF AS [UF RG], 
    dbo.SRA010.RA_CIC AS CPF, 
    dbo.SRA010.RA_PIS AS PIS, 
    dbo.SRA010.RA_NUMCP AS CTPS, 
    dbo.SRA010.RA_SERCP AS [Série CTPS], 
    dbo.SRJ010.RJ_FUNCAO AS [Cód Função], 
    dbo.SRJ010.RJ_DESC AS Função, 
    dbo.SRJ010.RJ_CODCBO AS CBO, 
    dbo.SRA010.RA_SITFOLH AS Status, 
    dbo.SRA010.RA_CC AS [Cód CC], 
    dbo.CTT010.CTT_DESC01 AS [Descrição Centro de Custo], 
    dbo.SRA010.RA_SINDICA AS [Cód Sindicato], 
    dbo.RCE010.RCE_DESCRI AS [Descrição Sindicato], 
    dbo.SRA010.RA_TNOTRAB AS [Cód Turno], 
    dbo.SR6010.R6_DESC AS [Descrição Turno], 
    dbo.SRA010.RA_REGRA AS Regra, 
    dbo.SRA010.RA_HRSMES AS [Carga Horaria], 
    dbo.SRA010.RA_BCDEPSA AS [Banco/Agencia], 
    dbo.SRA010.RA_CTDEPSA AS Conta, 
    dbo.SRA010.RA_ENDEREC AS Endereço, 
    dbo.SRA010.RA_NUMENDE AS Número, 
    dbo.SRA010.RA_BAIRRO AS Bairro, 
    dbo.SRA010.RA_MUNICIP AS Munícipio, 
    dbo.SRA010.RA_CEP AS CEP, 
    dbo.SRA010.RA_SEXO AS Sexo, 
    dbo.SRA010.RA_DDDCELU AS [Cel DDD], 
    dbo.SRA010.RA_NUMCELU AS Celular, 
    dbo.SRA010.RA_DEPTO AS [Cód Depto], 
    dbo.SQB010.QB_DESCRIC AS [Descrição Depto], 
    CASE dbo.SRA010.RA_DEFIFIS 
        WHEN '2' THEN 'Não' 
        ELSE 'Sim' 
    END AS [Deficiente?], 
    CASE dbo.SRA010.RA_TPDEFFI 
        WHEN '0' THEN 'Não Possui' 
        WHEN '1' THEN 'Física' 
        WHEN '2' THEN 'Auditiva' 
        WHEN '3' THEN 'Visual' 
        WHEN '4' THEN 'Intelectual' 
        WHEN '5' THEN 'Múltipla' 
        WHEN '6' THEN 'Reabilitado' 
        ELSE 'Não Possui' 
    END AS [Tipo Deficiência], 
    dbo.SRA010.RA_ADCCONF AS [C Confiança], 
    dbo.SRA010.RA_ADCTRF AS [Adc Transf], 
    CASE dbo.SRA010.RA_BHFOL 
        WHEN 'N' THEN 'Não' 
        WHEN 'S' THEN 'Sim' 
        ELSE 'Não' 
    END AS [Banco de Horas?], 
    dbo.SRA010.RA_ADMISSA
FROM 
    ((((dbo.SRA010 
    INNER JOIN dbo.SRJ010 ON dbo.SRA010.RA_CODFUNC = dbo.SRJ010.RJ_FUNCAO) 
    INNER JOIN dbo.CTT010 ON dbo.SRA010.RA_FILIAL = dbo.CTT010.CTT_FILIAL AND dbo.SRA010.RA_CC = dbo.CTT010.CTT_CUSTO) 
    INNER JOIN dbo.RCE010 ON dbo.SRA010.RA_SINDICA = dbo.RCE010.RCE_CODIGO) 
    INNER JOIN dbo.SR6010 ON dbo.SRA010.RA_TNOTRAB = dbo.SR6010.R6_TURNO) 
    INNER JOIN dbo.SQB010 ON dbo.SRA010.RA_DEPTO = dbo.SQB010.QB_DEPTO
WHERE 
    dbo.SRA010.RA_SITFOLH <> 'D' 
    AND dbo.SRA010.RA_ADMISSA BETWEEN 20240201 AND 20240228
    AND dbo.SRJ010.D_E_L_E_T_ <> '*' 
    AND dbo.CTT010.D_E_L_E_T_ <> '*' 
    AND dbo.RCE010.D_E_L_E_T_ <> '*' 
    AND dbo.SR6010.D_E_L_E_T_ <> '*' 
    AND dbo.SQB010.D_E_L_E_T_ <> '*'
ORDER BY 
    dbo.SRA010.RA_MAT;"""

In [3]:
#Conectar ao banco do LTOTVS
def conect_totvs():
    serv = '192.168.0.236'
    databa = 'PROTHEUS_PRODUCAO'
    user = 'totvs'
    passa = 'totvsip'
    conn = pymssql.connect(serv, user, passa, databa)
    cursor = conn.cursor(as_dict=True)
    return conn, cursor

con_tot, cursor = conect_totvs()

In [7]:
df = pd.read_sql_query(query, con_tot)

In [10]:
cemig = df[df['Descrição Centro de Custo'].str.contains('CEMIG')]